snscrape --jsonl --since 2021-05-01 twitter-hashtag "انتخابات until:2021-05-27" > election.json


In [1]:
filename = 'election'
hashtag = 'انتخابات'
min_word_length = 2

width = 800
height = 500
background_color = 'white'
min_font_size = 10
prefer_horizontal = 1.

In [2]:
%%capture
!pip install stopwords_guilannlp
!pip install hazm
!unrar e $filename
!pip install wordcloud-fa

from stopwords_guilannlp import stopwords_output
import re
from hazm import *
import json
from wordcloud_fa import WordCloudFa

In [3]:
stopwords = stopwords_output("Persian", output="set")
normalizer = Normalizer()

def preprocess(text):
  text = text.replace('\u200c', ' ')
  text = text.replace('<br/>', '\n')
  text = text.replace('\n', ' ')
  text = normalizer.normalize(text)
  text = re.sub(r"[^آابپتثجچحخدذرزژسشصضطءئظعغفقکگلمنوهی]+", " ", text)
  text = text.strip()
  tokens = text.split()
  tmp = []
  for token in tokens:
    if token not in stopwords and len(token) > min_word_length and token != hashtag:
      tmp.append(token)
  text = " ".join(tmp)
  return text

text = ""

with open(filename + '.json', 'rb') as f:
  for line in f:
    try:
      tweet = json.loads(line)
      text += preprocess(tweet['content']) + ' '
    except:
      pass

In [4]:
wordcloud = WordCloudFa(width=width, height=height,
                        background_color=background_color,
                        stopwords=stopwords,
                        min_font_size=min_font_size,
                        prefer_horizontal=prefer_horizontal)
wc = wordcloud.generate(text)
image = wc.to_image()
image.show()
image.save('wordcloud.png')